# Django ORM - Praca z danymi (komendy DML i DQL)

**Definicje modelu**

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'intro.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
from orm_app.models import Task

## R - Read (CRUD) - DQL (Data Query Language) cd

### Składanie warunków (operator `AND`)

Wypisywanie kolejnych warunków po przecinku, jako kolejnych parametrów funkcji filter powoduje składanie tych warunków (koniunkcje/iloczyn logiczną) - operator and.

Znajdż wszystkie wpisy w tabeli Task, które zaczynają się na Pr **i** kończą na anie.

In [ ]:
# Koniunkcja (and)

# metoda I
tasks = Task.objects.filter(name__startswith="Pr").filter(name__endswith="anie")

# metoda II
tasks = Task.objects.filter(name__startswith="Pr", name__endswith="anie")
print(tasks)

Z dokładnością do tego, że przy podawaniu parametrów po przecinku (metoda II) nie możemy dwa razy użyć parametru o tej samej nazwie.

In [ ]:
# nie można dwa razy użyć tego samego warunku
tasks = Task.objects.filter(name__startswith="Pr", name__startswith="T")

Alternatywę warunków (`or`) możemy uzyskać poprzez użycie operatora `|` na rozdzielnych querysetach.

In [ ]:
queryset1 = Task.objects.filter(name__startswith="Pr")
queryset2 = Task.objects.filter(name__endswith="anie")

tasks = queryset1 | queryset2
tasks

Zaprzeczenia warunków w bezpośredni sposób nie otrzymamy. Możemy kombinować z użyciem metod przeciwnych, na przykład zamiast metody `filter` możemy użyć metody `exclude`, ale bezpośredniej metody nie ma. Przynajmniej dopóki nie użyjemy obiektu Q.

### 3. Składanie zapytań za pomocą obiektów Q

**Q-objects** pozwalają nam na:
- składania operacji logicznych (operatory `or`, `and` i `not`)
- enkapsulację złożonych lookupów

Obiekt Q służy do enkapsulacji pojedynczego lookupu, ew. grupy lookupów.

In [ ]:
from django.db.models import Q

In [ ]:
Q(name__endswith='anie')

Następnie taki pojedynczy lookup, zamknięty w obiekcie Q może być składany z innym obiektem Q za pomocą operatorów `&` (and), `|` (or) ew. `^` (not).

In [ ]:
Q(name__endswith='anie') & Q(name__startswith="Pr")

In [ ]:
Q(name__endswith='anie') | Q(name__startswith="Pr")

In [ ]:
~(Q(name__endswith='anie') & Q(name__startswith="Pr"))

i nie ma ograniczenia na powtórzającą się nazwę parametru

In [ ]:
Q(name__startswith='Pr') & Q(name__startswith="T")

Otrzymane złożenie możemy wykorzystać w dowolnej z metod manadżera, która wykorzystuje lookupy.

In [ ]:
# Koniunkcja (and) z użyciem obiektów Q - operator &
result = Task.objects.filter(Q(name__endswith='anie') & Q(name__startswith="Pr"))
result

In [ ]:
# Alternatywa (or) z użyciem obiektów Q - operator |
result = Task.objects.filter(Q(name__endswith='anie') | Q(name__startswith="Pr"))
result

In [ ]:
# Zaprzeczenie (not) z użyciem obiektów Q - operator ~
result = Task.objects.filter(~(Q(name__endswith='anie') & Q(name__startswith="Pr")))
result

Enkapsulacja lookupów pozwala nam na tworzenie złożonych, ale wciąż czytelnych zapytań.

In [ ]:
pr_anie = Q(name__endswith='anie') | Q(name__startswith="Pr")
id_lt_3 = Q(id__lt=3)
mask = pr_anie & id_lt_3

Task.objects.exclude(mask)  # wykluczamy (`exclude`)

### Sortowanie - metoda `order_by` (klauzula `ORDER BY`)

Klauzula ORDER BY implementowana jest w Django przez metodę order_by klasy QuerySet. Metoda order_by jako parametr przyjmuje nazwę kolumny, po której dane mają zostać posortowane. Może przyjąć więcej niż jeden parametr, wtedy wpisy które mają identyczną wartość w pierwszej kolumnie będą sortowane po drugiej wpisanej kolumnie, itd. Wartości w kolumnach tekstowych sortowane są alfabetycznie, a wartości w kolumnach liczbowych numerycznie.

Posortujemy wpisy w tabeli Task po wartościach w kolumnie name.

In [ ]:
tasks = Task.objects.order_by('name')
for task in tasks:
    print(f"{task.id} {task.name}")

In [ ]:
# sql?
print(tasks.query)

Posortujmy wpisy w tabeli Task po kolumnie name, a wpisy które mają identyczną wartość w kolumnie name po kolumnie id.

In [ ]:
tasks = Task.objects.order_by('name', 'id')
for task in tasks:
    print(f"{task.id} {task.name}")

#### Sortowanie odwrotne - Metoda I (metoda `reverse`)

Sortować w kolejności odwrotnej możemy z użyciem metody reverse()

In [ ]:
tasks = Task.objects.order_by('name').reverse()
for task in tasks:
    print(f"{task.id} {task.name}")

In [ ]:
print(tasks.query)

#### Sortowanie odwrotne - Metoda II (nazwa kolumny)

lub poprzez wstawienie przed nazwą kolumny minusa.

In [ ]:
tasks = Task.objects.order_by('-name')
for task in tasks:
    print(f"{task.id} {task.name}")

### Funkcje agregujące (metoda `aggregate`)

Funkcje agregujące w sql to operatory, które wykonują na wskazanych danych proste statystyki takie jak średnia, wartość maksymalna, suma czy liczba wpisów (SUM, AVG, MIN, MAX, COUNT).

Najprostszym możliwym agregatem są zliczenia. W Django mamy na to dedykowaną metodę queryseta - `count`.

In [ ]:
Task.objects.all().count()

In [ ]:
Task.objects.filter(name__startswith="s").count()

W Django istnieje druga metoda za pomocą, której możemy zrobić to samo - `aggregate`.

In [ ]:
from django.db.models import Count

Task.objects.all().aggregate(Count('name'))

Metoda `aggregate` ma znacznie większe możliwości. Jako parametr przyjmuje instancje odpowiedniej klasy z modułu django.db.models (Sum, Avg, Min, Max, Count, ...). Te klasy reprezentują wbudowane w Django funkcje agregujące. Jako parametr przyjmują nazwę kolumny (atrybutu), na której ma zostać wyliczona ta funkcja.

Znajdź sumę indeksów wszystkich wpisów tabeli Task. 

In [ ]:
from django.db.models import Sum

a_sum = Task.objects.aggregate(Sum('id'))
print(a_sum)

In [ ]:
print(connection.queries[-1])

Funkcja agregująca zwraca słownik z kluczem 
`<nazwa_kolumny_wzdluz_ktorej_zachodzi_agregacja>_ _<nazwa_funkcji_agregującej>` oraz wartością będąca wyliczoną statystyką.

Jeżeli nazwa nam nie odpowiada możemy użyć aliasów.

In [ ]:
a_sum = Task.objects.aggregate(total=Sum('id'))
print(a_sum)

In [ ]:
print(connection.queries[-1])

Znajdź średnią wartość indeksu wpisów tabeli Task o wartości w kolumnie name "Szukanie"

In [ ]:
from django.db.models import Avg

avg = Task.objects.filter(name="Szukanie").aggregate(Avg('id'))
print(avg)

W jednym zapytaniu możemy umieścić kilka statystyk.

In [ ]:
from django.db.models import Avg, Count, Sum

stats = Task.objects.filter(name="Szukanie").aggregate(
    Avg('id'), Count('name'), Sum('id')
)
print(stats)

Popatrzmy co jeszcze można znaleźć w module `django.db.models`.

In [ ]:
import django.db.models as django_models

print(dir(django_models))  #  Count, Sum Max, Min, Sum, Avg, StdDev, Variance

### Metoda `annotate`

When we annotate our model in Django we are adding a summary record or some sort of aggregated record for every single model in the queryset. So rather than a single output number we are annotating every Django model in the queryset with a particular value that's not defined by default on that model

Podstawowa różnica pomiędzy metodami `aggregate` i `annotate` polega na tym, że kiedy używamy metody `annotate` dodajemy jakieś pole (wartość) do każdego wpisu z naszego queryseta, podczas gdy w metodzie `aggregate` wyliczamy jedną wartość na podstawie wszystkich wpisów z queryseta. Zamiast jednego wyniku liczbowego, anotujemy każdy model w querysecie z określoną wartością, która domyślnie nie jest zdefiniowana w tym modelu.

Metoda `annotate` jest stosowana na każdym wpisie z queryseta i dla każdego wpisu zwraca jakąś wartość. Pozwala na dodanie dodatkowych informacji do naszych wpisów. 

Założmy, że wyciągamy wszystkie taski i wynik chcemy uzupełnić o liczbę znaków w nazwie każdego z tasków.

Metody annotate używamy podobnie jak metody aggregate. Korzysta z funkcji z modułu `django.db.models.functions`.

In [ ]:
import django.db.models.functions

print(dir(django.db.models.functions))

Nas interesuje funkcja `Length`

In [ ]:
from django.db.models.functions import Length

tasks = Task.objects.annotate(len_name=Length('name'))

In [ ]:
print(tasks.query)

In [ ]:
tasks.first().len_name

Anotacji możemy używać w filtrach.

In [ ]:
tasks = Task.objects.annotate(len_name=Length('name')).filter(len_name__gte=10)
print(tasks)

Kilka innych przykładów.

In [ ]:
from django.db.models.functions import Upper, Concat
from django.db.models import Value
from django.db.models import CharField

concatenation = Concat(
    'name', Value(' [id='), 'id', Value(']'),
    output_field=CharField()
)
tasks = Task.objects.annotate(
    upper_name=Upper('name'), 
    message=concatenation
)
print(tasks)

W przypadku użycia różnych typów danych w funkcji `Concat` należy wskazać typ wyjściowy za pomocą parametru `output_field`.

In [ ]:
print(tasks.query)

In [ ]:
tasks.first().upper_name

In [ ]:
tasks.first().message

## U - UPDATE (CRUD) - DML (Data Manipulation Language)

### Klauzula UPDATE

In [ ]:
# Metoda I - metoda update mandżera modelu (i QuerySet-a)
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE ZMIENIĆ WSZYSTKICH wpisów 
# w tabeli.

tasks = Task.objects.filter(name__endswith="enie").update(name="GGotowanie")
print(tasks)

Widzimy, że metoda update nie zwraca nam obiektu klasy QuerySet tylko liczbę zmodyfikowanych wpisów.

In [ ]:
# Metoda II - bezpośrednia modyfikacja wartości atrybutu instancji modelu

task = Task.objects.get(name="Dodawanie")
task.name = "Odejmowanie"
task.save()

A w jaki sposób możemy sprawdzić jakie zapytanie zostało wykonane na bazie, jeżeli nie mamy dostępu do obiekty queryset?

Możemy użyć obiektu `connection` z modułu `django.db`.

In [ ]:
from django.db import connection

connection.queries

Co prawda w ten sposób dostaniemy wszystkie zapytania wykonane w ramach nawiązanego polączenia, ale interesujące nas zapytanie będzie wśród nich. Najnowsze zapytanie będzie ostatnie na liście.

In [ ]:
connection.queries[-1]

## D - DELETE (CRUD) - DML (DATA Manipulation Language)

### Klauzula DELETE

In [ ]:
# Metoda I - metoda delete menadżera modelu (i QuerySet-a)
# UWAGA! Dane należy najperw przefiltrować, żeby jednym zapytanie NIE USUNĄĆ WSZYSTKICH wpisów 
# w tabeli.

task = Task.objects.filter(name="Pływanie").delete()
print(task)

Podobnie jak metoda update, metoda delete nie zwraca nam obiektu klasy Queryset tylko informacje o liczbie usuniętych wpisów (tym razem w postaci tupli, której pierwszy element to całkowita liczba usuniętych wpisów, a drugi element to słownik z kluczami będącymi nazwami modelu i wartościami będącymi liczbą usuniętych w danym modelu wpisów).

In [ ]:
# Metoda II - metoda delete instancji modelu

task = Task.objects.get(name="Odejmowanie")
task.delete()

Metoda delete instancji modelu zwraca nam identyczną odpowiedź co metoda delete menadżera modelu (i Queryset-a)